## Modelling - ML

The goal for our work is to not only train a model to identify toxic comments, but to do so while reducing bias.
Bias in this task can be viewed as the situation where certain identies such as 'Black', 'Muslim', 'Gay' e.t.c, begin triggering toxic classification for comments they are in, even when the comment is actually positive. This is a key issue in toxic comment classification. 

The goal of the [jigsaw unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data) Kaggle challenge was to reduce this bias via a newly developed submetric which we have defined below.

**Note: The goal of the model is simply to predict the toxicity score of a model.** 

The bias weighted ROC metric below is calulated by taking segmenting the the dataset into identity subgroups by using the provided identity labels and then calculating the subgroup metrics. 

### Metrics:

In addition to accuracy we will observe the below metrics for our models

#### Overall ROC-AUC:

This is the standard ROC-AUC for the full evaluation set. In other words this is the area under the Reciever Operating Characteristic curve. It compares the true positive and false positive rates of a binary model.

#### Subgroup ROC-AUC:

Here, we restrict the data set to only the examples that mention the specific identity subgroup. A low value in this metric means the model does a poor job of distinguishing between toxic and non-toxic comments that mention the identity.

#### BPSN AUC:

BPSN (Background Positive, Subgroup Negative) AUC: Here, we restrict the test set to the non-toxic examples that mention the identity and the toxic examples that do not. A low value in this metric means that the model confuses non-toxic examples that mention the identity with toxic examples that do not, likely meaning that the model predicts higher toxicity scores than it should for non-toxic examples mentioning the identity.

#### BNSP AUC:

BNSP (Background Negative, Subgroup Positive) AUC: Here, we restrict the test set to the toxic examples that mention the identity and the non-toxic examples that do not. A low value here means that the model confuses toxic examples that mention the identity with non-toxic examples that do not, likely meaning that the model predicts lower toxicity scores than it should for toxic examples mentioning the identity.


#### Generalized Mean of Bias AUCs
To combine the per-identity Bias AUCs into one overall measure, we calculate their generalized mean as defined below:

$M_p(m_s) = \left(\frac{1}{N} \sum_{s=1}^{N} m_s^p\right)^\frac{1}{p}$

Where:

$M_p$ = the $p$th power-mean function

$m_s$ = the bias metric $m$ calulated for subgroup $s$

$N$ = number of identity subgroups

For this competition, JigsawAI use a p value of -5 to encourage competitors to improve the model for the identity subgroups with the lowest model performance.

### Final Metric
We combine the overall AUC with the generalized mean of the Bias AUCs to calculate the final model score:

$score = w_0 AUC_{overall} + \sum_{a=1}^{A} w_a M_p(m_{s,a})$

$A$ = number of submetrics (3)

$m_{s,a}$ = bias metric for identity subgroup $s$ using submetric $a$

$w_a$ = $a$ weighting for the relative importance of each submetric; all four $w$ values set to 0.25


### Process:

#### Classical ML models
This is primarily an NLP task, our X feature matrix will be based off the text from online comments. We have defined a pre-processing pipeline in the 'preprocessing.ipynb' notebook to use for our our ML classifiers and a seperate pre-processing pipeline for the neural network models we are planning on training.

From the classic ML classifer models, we intend to use the following models - our base word embedding technique will be TF-IDF: 

   * Logistic Regression
   * SVM
   * Random Forest
 
   
We will carry out hyperparameter optimization for each model and calculate the metrics for each.

#### Neural Networks

We will also train a neural network to answer this problem. We will start with a basic LSTM model which will be made of:
    
   * Two LSTM layers to read through the data
   * Two Dense layers w/ 4 nodes
   * Output layer using sigmoid for the classes
   
We will then seek to improve this LSTM by creating a Bidrectional LSTM (BiLSTM) which we believe will improve accuracy by reading input sequences in both directions. If time allows we will also attempt to include a simple attention mechanism.

The NN models will use Glove 840B 300d word embeddings. 


In [131]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import contractions
import tqdm
from tqdm import tqdm
tqdm.pandas()
import gc
import operator

import warnings
warnings.filterwarnings('ignore')

## Stage 1: Apply pre-processing to text

In [4]:
# Load train_data
train_df = pd.read_csv('data/train_clean.csv')

# Load in test data
test_df = pd.read_csv('data/test_clean.csv')

In [187]:
test_df.rename({'toxicity':'target'}, axis=1, inplace=1)

In [188]:
test_df['target']

0         0
1         0
2         0
3         0
4         1
         ..
194635    0
194636    0
194637    0
194638    0
194639    0
Name: target, Length: 194640, dtype: int64

# drop uneeded columns as well

In [189]:
train_df = train_df.iloc[:,1:]
test_df = test_df.iloc[:,1:]

In [190]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
import contractions
import string

def text_cleaner(df, col_name, clean_col_name):
   

    # Lemmatize functions to be called lated
    # Lemmatize nouns
    def lemmatize_text_noun(text):
        return [lemmatizer.lemmatize(w, pos='n') for w in text]
    
    # Lemmatize verbs
    def lemmatize_text_verb(text):
        return [lemmatizer.lemmatize(w, pos='v') for w in text]
    # Lemmatize adjectives
    def lemmatize_text_adj(text):
        return [lemmatizer.lemmatize(w, pos='a') for w in text]

    # Lemmatize adverbs
    def lemmatize_text_adv(text):
        return [lemmatizer.lemmatize(w, pos='r') for w in text]
    
    # Expand contraction method
    def contraction_expand(text):
        return contractions.fix(text)
    
    # To lower case.
    df[clean_col_name] = df[col_name].apply(lambda x: x.lower())
    
    # Expand contractions
    df[clean_col_name] = df[clean_col_name].apply(lambda x: contraction_expand(x))
    
    #Tokenize:
    tokenizer = TweetTokenizer(reduce_len=True)
    df[clean_col_name] = df[clean_col_name].apply(lambda x: tokenizer.tokenize(x))
   
    
    #Remove Stop words
    stop_words = stopwords.words('english')
    df[clean_col_name] = df[clean_col_name].apply(lambda x: [item for item in x if item not in stop_words])
    
    #Delete punctuation
    punc_table = str.maketrans('', '', string.punctuation)
    df[clean_col_name] = df[clean_col_name].apply(lambda x: [item.translate(punc_table) for item in x])
    
    # LEMMATIZATION
    lemmatizer = WordNetLemmatizer()
    
    df[clean_col_name] = df[clean_col_name].apply(lemmatize_text_noun)
    df[clean_col_name] = df[clean_col_name].apply(lemmatize_text_verb)
    df[clean_col_name] = df[clean_col_name].apply(lemmatize_text_adj)
    df[clean_col_name] = df[clean_col_name].apply(lemmatize_text_adv)
    
    
    return None
def detokenizer(df, col_name):
    detokenizer = TreebankWordDetokenizer()
    df[col_name+'_detokenize'] = df[col_name].apply(lambda x: detokenizer.detokenize(x))
    
    return None

In [7]:
%%time
# Run the cleaner func on train data
text_cleaner(train_df, 'comment_text', 'comment_text_clean')

CPU times: user 21min 8s, sys: 24.1 s, total: 21min 32s
Wall time: 25min 42s


In [179]:
%%time 
# Run the same cleaner on the training data
text_cleaner(test_df, 'comment_text', 'comment_text_clean')

CPU times: user 2min 15s, sys: 21.8 s, total: 2min 37s
Wall time: 2min 57s


In [8]:
%%time
detokenizer(train_df,'comment_text_clean')

CPU times: user 3min 35s, sys: 454 ms, total: 3min 35s
Wall time: 3min 36s


In [180]:
%%time
# detokenize the test data
detokenizer(test_df, 'comment_text_clean')

CPU times: user 23.1 s, sys: 143 ms, total: 23.2 s
Wall time: 23.4 s


In [26]:
train_df['comment_text_clean_detokenize']

0          cool  like   would want mother read    really ...
1          thank   would make life lot le anxietyinducing...
2              urgent design problem  kudos take  impressive
3                     something I able install site  release
4                                       haha guy bunch loser
                                 ...                        
1804869    maybe tax  thing  would collect product import...
1804870         call people still think divine role creation
1804871                thank    right wrong    follow advice
1804872    anyone quote follow exchange  even apocryphal ...
1804873    student define ebd legally disable eligible sp...
Name: comment_text_clean_detokenize, Length: 1804874, dtype: object

In [181]:
test_df['comment_text_clean_detokenize']

0         jeff session another one trump orwellian choic...
1         actually inspect infrastructure grand chief st...
2         wishful think democrat fault  100 th time  wal...
3         instead wring hand nibble periphery issue  fac...
4         many commenters garbage pile high yard  bald t...
                                ...                        
194635      lose job promote misinformation harmful student
194636    thin project mean low fire danger improve wild...
194637            hope millennials happy put airhead charge
194638    I think kellyanne conway   k   trump whisperer...
194639    still figure pizza ak cost pizza washington  i...
Name: comment_text_clean_detokenize, Length: 194640, dtype: object

## Stage 2: Modelling

The data has been pre-processed for our models. We can now begin model training.

One thing to note is that the usual train-validation split is different for this case. In order to calculate the sub-group AUC metrics, we need the identity label data that is contained in the dataframe. So, what we will do here is create a training dataframe and a validation data frame using scikit-learn's *train_test_split function*, we will then create the *X* feature matrix (which will be the comments) and the *y* target matrix (the toxicity label) from these two dataframes.

After we have fitted a model and predicted results, we can then append the predictions to the dataframe and calculate the ROCs. 

#### Defining subgroup ROC metrics

In [244]:
from sklearn import metrics
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df.loc[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df.loc[~df[subgroup] & df[label]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df.loc[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df.loc[~df[subgroup] & ~df[label]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset.loc[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)



In [245]:
def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)
    


In [192]:
# set the y values
y_train = train_df['target']
y_test = test_df['target']

# we will directly choose the comment feature column in the tfid vectorizer and the models

print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')
      

y_train shape: (1804874,)
y_test shape: (194640,)


### Logistic Regression

We will use Grid Search Cross validation to optimize our hyperparameters on the training set. We will then fit the best estimator and then predict on the test set and calculate relevent metrics.

Because the subgroup ROCs are calculated post prediction and require predictions to be appended to a dataframe, we cannot actually pass the metrics into the scoring function of scikit-learn's gridsearchCV(). Instead we will just test the models on accuracy and total ROC-AUC and then refitting the grid-search on the model with the best ROC-AUC as a proxy.

In [ ]:
%%time
#no scaling data is already transformed

# Instantiate the vectorizer, we will pass in the TweetTokenizer() from nltk 
tfid_vec_2 = TfidfVectorizer(lowercase=False, tokenizer = tweet_tokenizer.tokenize)

# define pipeline
pipeline = Pipeline([('tf-idf', tfid_vec_2), 
                     ('model', SVC())])

# Define scoring functions
scorers = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}

# define parameters
param_grid_log = [{'model': [LogisticRegression()], 'tf-idf': [tfid_vec_2], 
                   'model__penalty': ['l1', 'l2'],
                   'model__C': [.001, 0.01, 0.1, 1, 10, 100]}]

# Setting refit='AUC', refits an estimator on the whole dataset with the
# parameter setting that has the best cross-validated AUC score, i.e it finds the params that gives the best scores
# then refits using the ones that give the best AUC. 
grid_log = GridSearchCV(pipeline, param_grid_log, cv=5, scoring=scorers, refit='AUC', 
                        return_train_score=True, n_jobs=-1)

fittedgrid = grid_log.fit(train_df['comment_text_clean_detokenize'], train_df['target'])


In [ ]:
from sklearn.externals import joblib

# Save best estimator to file
joblib.dump(fittedgrid.best_estimator_, 'saved_models/best_log_reg.pkl')

In [241]:
# Get scores
train_accuracy = fittedgrid.score(train_df['comment_text_clean_detokenize'], y_train)
test_accuracy = fittedgrid.score(test_df['comment_text_clean_detokenize'], y_test)

In [193]:
# predict train and test values
y_train_pred = fittedgrid.predict(train_df['comment_text_clean_detokenize'])
y_test_pred = fittedgrid.predict(test_df['comment_text_clean_detokenize'])

y_train_pred_prob = fittedgrid.predict_proba(train_df['comment_text_clean_detokenize'])
y_test_pred_prob = fittedgrid.predict_proba(test_df['comment_text_clean_detokenize'])

In [195]:
# Append preds and probabilities to train and valid dfs


train_df['Prediction_log'] = y_train_pred
train_df['Prediction_probability_log'] = y_train_pred_prob[:, 1]
test_df['Prediction_log'] = y_test_pred
test_df['Prediction_probability_log'] = y_test_pred_prob[:, 1]

In [196]:
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']



In [234]:
for col in identity_columns + ['target']:
    train_df[col] = np.where(train_df[col] >= 0.5, True, False)
    test_df[col] = np.where(test_df[col] >= 0.5, True, False)

In [236]:
# Generate the AUC metrics 

SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'

MODEL_NAME = 'Prediction_log'
TOXICITY_COLUMN = 'target'

log_bias_metrics_df_train = compute_bias_metrics_for_model(train_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
log_final_metric_train = get_final_metric(log_bias_metrics_df_train, calculate_overall_auc(train_df, MODEL_NAME))

log_bias_metrics_df_test = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
log_final_metric_test = get_final_metric(log_bias_metrics_df_test, calculate_overall_auc(test_df, MODEL_NAME))

0.7150250269924768

In [237]:
bias_metrics_df_train

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
5,muslim,21006,0.677591,0.721540,0.700603
3,christian,40423,0.678946,0.741414,0.682061
2,homosexual_gay_or_lesbian,10997,0.683245,0.701611,0.725101
4,jewish,7651,0.685615,0.729084,0.699830
6,black,14901,0.694250,0.697453,0.740054
7,white,25082,0.697999,0.707779,0.733958
1,female,53429,0.719074,0.735354,0.727654
0,male,44484,0.722769,0.731714,0.734609
8,psychiatric_or_mental_illness,4889,0.723606,0.729101,0.737448


In [238]:
bias_metrics_df_test

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
2,homosexual_gay_or_lesbian,1065,0.657019,0.695304,0.707064
3,christian,4226,0.667762,0.742481,0.671798
5,muslim,2040,0.671500,0.720456,0.697212
4,jewish,835,0.677827,0.723069,0.699662
6,black,1519,0.678529,0.691751,0.731926
7,white,2452,0.688773,0.705066,0.729291
0,male,4386,0.712445,0.732588,0.725413
1,female,5155,0.714756,0.737360,0.723030
8,psychiatric_or_mental_illness,511,0.720370,0.721739,0.743106


In [243]:
print(f'train_accuracy:{train_accuracy}')
print(f'train weighted subgroup AUC:{get_final_metric(bias_metrics_df_train, calculate_overall_auc(train_df, MODEL_NAME))}')
print(f'test_accuracy:{test_accuracy}')
print(f'test weighted subgroup AUC::{get_final_metric(bias_metrics_df_val, calculate_overall_auc(test_df, MODEL_NAME))}')


train_accuracy:0.9538283994514587
train weighted subgroup AUC:0.7197198017038468
test_accuracy:0.9496323269141005
test weighted subgroup AUC::0.7150250269924768


#### Model Evaluation:

The logistic regression performed well on pure accuracy, with a train accuracy of 95.38% and 94.96% test accuracy. What is also positive to see is that our hyperparameter optimization has led to a model which does not overfit excessively. 

However when we look at the weighted subgroup AUC metric, the 71.9% train score and 71.5% test score show that the model did have a tendency towards biased predictions for certain subgroup. In comparison, the benchmark CNN that was provided had a weighted AUC score of 88.35% albeit just on a validation set 


For example we can see that for the 'black' identity BPSN AUC was relatively low, suggesting the model is likely overweighting mentions of the 'black' identity with toxicity. 

--------------

### SVM

We will now carry out a similar process with SVM to see if this performs appreciably different to logistic regression. 

In [252]:
%%time
#NOW DO FOR SVC

param_grid_svc = [{'model__kernel': ['rbf'],'tf-idf': [tfid_vec_2],
              'model__C': [0.01, 0.1, 1, 10], 'model__probability': [True]}]

grid_svc = GridSearchCV(pipeline, param_grid_svc, scoring=scorers, cv=5, refit='AUC',
                       return_train_score=True, n_jobs=-1, verbose=1)

fittedgrid_svc = grid_svc.fit(train_df['comment_text_clean_detokenize'], train_df['target'])

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [253]:
# Save best estimator to file
joblib.dump(fittedgrid_svc.best_estimator_, 'saved_models/best_svm.pkl')

NameError: name 'fittedgrid_svc' is not defined

In [ ]:
# Get scores
train_accuracy = fittedgrid_svc.score(train_df['comment_text_clean_detokenize'], y_train)
test_accuracy = fittedgrid_svc.score(test_df['comment_text_clean_detokenize'], y_test)

In [ ]:
# predict train and test values
y_train_pred = fittedgrid_svc.predict(train_df['comment_text_clean_detokenize'])
y_test_pred = fittedgrid_svc.predict(test_df['comment_text_clean_detokenize'])

y_train_pred_prob = fittedgrid_svc.predict_proba(train_df['comment_text_clean_detokenize'])
y_test_pred_prob = fittedgrid_svc.predict_proba(test_df['comment_text_clean_detokenize'])

In [ ]:
# Append preds and probabilities to train and valid dfs


train_df['Prediction_svc'] = y_train_pred
train_df['Prediction_probability_svc'] = y_train_pred_prob[:, 1]
test_df['Prediction_svc'] = y_test_pred
test_df['Prediction_probability_svc'] = y_test_pred_prob[:, 1]

In [ ]:
# Generate the AUC metrics 

SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'

MODEL_NAME = 'Prediction_svc'
TOXICITY_COLUMN = 'target'

svm_bias_metrics_df_train = compute_bias_metrics_for_model(train_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
svm_final_metric_train = get_final_metric(svm_bias_metrics_df_train, calculate_overall_auc(train_df, MODEL_NAME))

svm_bias_metrics_df_test = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
svm_final_metric_test get_final_metric(svm_bias_metrics_df_test, calculate_overall_auc(test_df, MODEL_NAME))

--------

### Random Forest

For our final model we will try the Random Forest Classifier which is an ensemble method. It is based on the Decision Tree model, only the Random Forest works by fitting on random sub samples of the data (with replacement), which is known as 'bagging'. A voting algorithm is then applied on the results of each of the trees to determine the final class of the data point.

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
# We control the model max depth and try a few different min_ leaf_counts
param_grid_RF = {'model': [RandomForestClassifier()], 'tf-idf': [tfid_vec_2]
              'model__max_depth': [10,15,20,30,50,100,None], 'model__min_samples_leaf': [2,5,10]
             }
grid_RF = GridSearchCV(pipeline, param_grid_RF, scoring=scorers, cv=5, refit='AUC',
                       return_train_score=True, n_jobs=-1)

fittedgrid_RF = grid_RF.fit(train_df['comment_text_clean_detokenize'], train_df['target'])

In [ ]:
# Save best estimator to file
joblib.dump(fittedgrid_RF.best_estimator_, 'saved_models/best_rf.pkl')

In [ ]:
# Get scores
train_accuracy = fittedgrid_RF.score(train_df['comment_text_clean_detokenize'], y_train)
test_accuracy = fittedgrid_RF.score(test_df['comment_text_clean_detokenize'], y_test)

In [ ]:
# predict train and test values
y_train_pred = fittedgrid_RF.predict(train_df['comment_text_clean_detokenize'])
y_test_pred = fittedgrid_RF.predict(test_df['comment_text_clean_detokenize'])

y_train_pred_prob = fittedgrid_RF.predict_proba(train_df['comment_text_clean_detokenize'])
y_test_pred_prob = fittedgrid_RF.predict_proba(test_df['comment_text_clean_detokenize'])

In [ ]:
# Append preds and probabilities to train and valid dfs


train_df['Prediction_RF'] = y_train_pred
train_df['Prediction_probability_RF'] = y_train_pred_prob[:, 1]
test_df['Prediction_RF'] = y_test_pred
test_df['Prediction_probability_RF'] = y_test_pred_prob[:, 1]

In [ ]:
# Generate the AUC metrics 

SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'

MODEL_NAME = 'Prediction_svc'
TOXICITY_COLUMN = 'target'

rf_bias_metrics_df_train = compute_bias_metrics_for_model(train_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
rf_final_metric_test = get_final_metric(rf_bias_metrics_df_train, calculate_overall_auc(train_df, MODEL_NAME))

rf_bias_metrics_df_test = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
rf_final_metric_test = get_final_metric(rf_bias_metrics_df_test, calculate_overall_auc(test_df, MODEL_NAME))

asses rf performance

--------

### Model performance

Now that we have trained three seperate models which have been shown to deliver strong performance in text classification tasks in the past let us take the time to compare them side-by-side and also discuss their short-comings in terms of reducing bias

In [ ]:
#Display the subgroup bias metrics tables for each model and then the final metrics for each model for test only
display(log_bias_metrics_df_test)
display(svm_bias_metrics_df_test)
display(rf_bias_metrics_df_test)

In [ ]:
# Display final metrics for each model for test only.
print(f' Final metric for Logistic Regression: {log_final_metric_test}')
print(f' Final metric for SVM: {svm_final_metric_test}')
print(f' Final metric for Random Forest: {rf_final_metric_test}')